this notebook keep various facts about linear model. By linear model, I mean a model $w_0, \mathbf{w} \in \mathbb{R}^{K}$ trying to minimize $\|y-X\mathbf{w} - w_0\|_2^2 + \phi(\mathbf{w})$, where $\phi(\mathbf{w})$ is some regularization on $\mathbf{w}$, but not $w_0$. Models such as ridge regression, lasso, etc. all fall into the definition of linear model here.

## How to deal with intercept

While in my textbooks, the intercept term is assumed to be zero for ridge regression or lasso, in practice, this can be an issue.

As it turns out, we can fit the intercept for different types of linear model using a single algorithm. That is, we first center $y$, $X$, then fit the model assuming intercept is zero, and then compute the intercept using a particular formula. This is the way `sklearn` handles intercept learning in probably all its linear models (at least by checking source code, it's true for ordinary least square, ridge regression, ElasticNet, and OMP).

The key to see this is that, suppose we have found the a pair of optimal $\mathbf{w}$ and $w_0$. Given $\mathbf{w}$, we can see that it's an ordinary least square problem for $w_0$. Then you can show that the optimal $w_0$ is the mean of $y-X\mathbf{w}$ over each sample point ($y-X\mathbf{w}$ is a vector of length same as number of sample points).

To compute the mean of $y-X\mathbf{w}$, we see that it can be done in the two ways, after constructing a matrix with $K+1$ columns, `y`, `-w1*X[:,0]`, `-w2*X[:,1]`, ..., `-wk*X[:,K-1]` (Python notation).

1. First sum this matrix over each row, then leaving a column vector, then take the mean of this column vector. This gives mean of $y-X\mathbf{w}$.
2. First sum this matrix over each column, then leaving a row vector, then take the sum of this row vector, divide by number of sample points.

Clearly these two are the same. Moreover, when all columns are centered (having zero mean), clearly intercept is zero, if we use the second method.

The bottom line is, when data is centered, the **optimal intercept must be zero**. Thus, we can constraint our feasible set to those without the added intercept.

Thus, in sklearn, all data are first centered, and then computed without mean, then adding back the mean.

We can show that this should give the correct result, because, given any $2(K+1)$ numbers $y_0, x_1, x_2, \ldots, x_K$, and $y_1, z_1, z_2, \ldots, z_K$ we can show that there are correspondences between solutions for $\|(y-y_0)-[X - (x_1, x_2, \ldots, x_K)]\mathbf{w} - w_0 \|_2^2 + \phi(\mathbf{w})$ and solutions for $\|(y-y_1)-[X - (z_1, z_2, \ldots, z_K)]\mathbf{w} - w_0\|_2^2 + \phi(\mathbf{w})$.

1. Basically, for any (not necessarily optimal) solution for first problem, there's a solution for the second problem, with same $\mathbf{w}$, and adjusted offset $w_0$ to account for differences in those $2(K+1)$ numbers, yielding same cost.
2. In our case, the first set of $K+1$ numbers are all zero. The second set are the means. Since we find the optimal for the second problem, there can't be a better solution for the first (original) problem, due to their correspondences. Notice that when solving the second problem, we keep intercept to zero, but this won't ignore the optimal one, as we have shown, **as long as our solver is exact** (which is true for many linear models, perhaps except OMP).


Notice that, in sklearn, most linear models also can do normalization. This won't give the same solution as the unnormalized one.